In [1]:
import sys, os
if os.path.abspath(os.pardir) not in sys.path:
    sys.path.insert(0, os.path.abspath(os.pardir))
import CONFIG
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import pydicom
import random

In [3]:
ROOT = CONFIG.CFG.DATA.BASE
BATCH_SIZE = 64

In [4]:
train_df = pd.read_csv(os.path.join(ROOT, "train.csv"))
train_df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

In [28]:
class OSICLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(OSICLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, X):
        out, _ = self.lstm(X)
        return out

In [33]:
model = OSICLSTM(4, 4, 1)
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [34]:
len(list(model.parameters()))

4

In [35]:
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

torch.Size([16, 4])
torch.Size([16, 4])
torch.Size([16])
torch.Size([16])


In [36]:
print(model)

OSICLSTM(
  (lstm): LSTM(4, 4, batch_first=True)
)


In [38]:
ALL_PATIENTS = train_df["Patient"].unique().tolist()

In [53]:
for patient in ALL_PATIENTS:
    model.zero

    patient_data = train_df[train_df["Patient"] == patient][['Weeks', 'FVC', 'Percent', 'Age']].values
    patient_data = torch.tensor(patient_data).float()
    for data in patient_data:
        out = model(data.view(1, 1, -1))
        print(out)
    break

tensor([[[ 0.0000e+00, -1.4214e-27,  0.0000e+00,  0.0000e+00]]],
       grad_fn=<TransposeBackward0>)
tensor([[[ 0.0000e+00, -3.0863e-25,  0.0000e+00,  0.0000e+00]]],
       grad_fn=<TransposeBackward0>)
tensor([[[ 0.0000e+00, -1.3886e-22,  0.0000e+00,  0.0000e+00]]],
       grad_fn=<TransposeBackward0>)
tensor([[[ 0.0000e+00, -8.6778e-24,  0.0000e+00,  0.0000e+00]]],
       grad_fn=<TransposeBackward0>)
tensor([[[ 0.0000e+00, -2.0733e-22,  0.0000e+00,  0.0000e+00]]],
       grad_fn=<TransposeBackward0>)
tensor([[[ 0.0000e+00, -1.7821e-22,  0.0000e+00,  0.0000e+00]]],
       grad_fn=<TransposeBackward0>)
tensor([[[ 0.0000e+00, -6.5509e-20,  0.0000e+00,  0.0000e+00]]],
       grad_fn=<TransposeBackward0>)
tensor([[[ 0.0000e+00, -4.8398e-20,  0.0000e+00,  0.0000e+00]]],
       grad_fn=<TransposeBackward0>)
tensor([[[ 0.0000e+00, -1.0439e-18,  0.0000e+00,  0.0000e+00]]],
       grad_fn=<TransposeBackward0>)


In [27]:
with torch.no_grad():
    input = torch.randn(2, 5, 4)
    print(input)
    print()
    out = model(input)
    print(out)
    print(out.shape)

tensor([[[-0.5860,  1.3153, -1.0502, -0.5227],
         [-1.1907, -1.2158, -0.4712, -0.1658],
         [-0.9553,  0.9396, -1.2907,  0.4846],
         [-0.1314, -0.4851,  0.1461,  0.1841],
         [ 0.1598, -0.7560,  0.1895,  0.5957]],

        [[-0.9853,  0.9842, -0.3083, -0.5643],
         [ 1.4160, -0.1827,  0.5166, -0.1190],
         [-0.0798, -0.3892, -0.3054,  0.2493],
         [-0.3142, -2.8067, -1.8554, -0.9012],
         [ 0.9378, -0.8356,  2.2742,  1.2010]]])

tensor([[[ 0.1448, -0.0468, -0.0903, -0.0212,  0.0090, -0.0891, -0.0880,
          -0.0046, -0.0033, -0.0360],
         [ 0.2230, -0.0832, -0.1429, -0.0101,  0.0148, -0.1451, -0.1288,
           0.0017,  0.0270, -0.0820],
         [ 0.2773, -0.0919, -0.1637, -0.0215,  0.0170, -0.1744, -0.1577,
          -0.0025,  0.0194, -0.1195],
         [ 0.2993, -0.0928, -0.1835, -0.0177,  0.0165, -0.1946, -0.1785,
          -0.0057,  0.0355, -0.1458],
         [ 0.3126, -0.0837, -0.1999, -0.0143,  0.0147, -0.2093, -0.1937,
        